In [1]:
import numpy as np
from sklearn.datasets import load_iris #testing purposes
from sklearn.neighbors import KNeighborsClassifier

In [2]:
X = load_iris().data
Y = load_iris().target
print(np.shape(X))
print(np.shape(Y))

(150, 4)
(150,)


In [3]:
train_X = X[0:125:,:]
train_y = Y[0:125]
test_X = X[125:,:] 
test_y = Y[125:]

In [4]:
np.shape(test_X)

(25, 4)

In [5]:
np.shape(train_X) # n = 125 , m = 25

(125, 4)

## Predicting labels (returns accuracy)

In [6]:
def predict(train_X,train_y,test_X,test_y,k,distance_type):
    '''
    train_X (nxp): design matrix of all training observations 
    train_y (nx1): vehicle class of each training observation. 
    test_X (mxp): design matrix of all testing observations
    k: number of neighbors to consider
    Note: 3 Minkowski distance types are considered: manhattan, euclidean, cubic.
    '''
    n,p = np.shape(train_X)
    m,p = np.shape(test_X)
    minimum_index = 0
    predicted_labels = list()
    dist = dict({'manhattan':1,'euclidean':2,'cubic':3})

    for i in range(m):                     # predicting class of 25 vectors 
        distance = np.zeros(n)             # distance of all 125 training vectors from test vector i
        neighbor_labels = list()
        for j in range(n):                 # finding distance of every training vector from test vector
            distance[j] = np.linalg.norm(test_X[i,:] - train_X[j,:],ord=dist[distance_type]) 
        ranked_distance = np.argsort(distance)                               # ranked_distance: indexes of n vectors sorted by increasing distance
        
        for l in ranked_distance[0:k]:
            neighbor_labels.append(train_y[l]) # finding labels of neighbors

        pred_label = max(set(neighbor_labels), key = neighbor_labels.count)  # choosing majority label
        predicted_labels.append(pred_label)                                  # appending majority label of ith observation

    accuracy_scratch = sum(test_y == predicted_labels) / len(test_y)

    return accuracy_scratch # returns prediction accuracy

In [7]:
pred_labels = predict(train_X,train_y,test_X,test_y,k=5,distance_type='manhattan')
pred_labels

0.8

In [8]:
# COMPARE WITH SKLEARN KNEIGHBORS

neigh = KNeighborsClassifier(n_neighbors=5,p=1)
neigh.fit(train_X, train_y)
pred_vec = np.ones(len(test_y))
for i in range(len(test_y)):
    pred_vec[i] = neigh.predict([test_X[i,:]]) # 84% accuracy

accuracy_sk = sum(test_y == pred_vec) / len(test_y)

In [9]:
accuracy_sk

0.8

## Tuning k

In [10]:
def k_tune(train_X,train_y,distance_type,folds):
    ''' The idea is to split training set into l parts. train on l-1 partitions and test on 1 partition.
        Then, find the mean accuracy of all l models for each value k (neighbors). 
        The optimal k will be found for all distance types. 

        input expectations:
            distance_type (string): manhattan, euclidean, or cubic.
            folds (int) 
            train_X (nxp numpy array) 
            train_y (nx1 numpy array) 

    '''
    kf = KFold(n_splits = folds, random_state = True, shuffle = True)
    neighbors = [1,3,5,7,9]
    accuracy_mat = list()
    mean_accuracy = list()

    for train_index, test_index in kf.split(train_X):       # Iterate through each of l folds for each lamda 
        temp_X_train = train_X[train_index]
        temp_y_train = train_y[train_index]
        temp_X_test = train_X[test_index]
        temp_y_test = train_y[test_index]

        accuracy_vec = np.zeros(len(neighbors))

        for k in range(len(neighbors)):
            accuracy_vec[k] = predict(temp_X_train,temp_y_train,temp_X_test,temp_y_test,neighbors[k],distance_type)
        accuracy_mat.append(accuracy_vec)
        
    accuracy_mat = np.array(accuracy_mat) 
    folds,neighbor = np.shape(accuracy_mat)
    for i in range(neighbor):
        accuracy = 0
        for j in range(folds):
            accuracy += accuracy_mat[j,i]
        mean_accuracy.append(accuracy / folds)
    max_value = max(mean_accuracy)
    max_index = mean_accuracy.index(max_value)         # contains the mean prediction accuracy for each k neighbors value.
    
    # returned: First object: best k neighbors. Second object: list of prediction accuracies for all values of k
    return neighbors[max_index], max_value

In [11]:
from sklearn.model_selection import KFold
optimal_k,max_value = k_tune(train_X,train_y,distance_type = 'cubic',folds=6)

In [12]:
optimal_k 
# Manhattan (L1 norm)
#     3-fold: k=3
#     4-fold: k=3
#     5-fold: k=3
#     6-fold: k=5 
# Euclidean (L2 norm)
#     3-fold: k=3
#     4-fold: k=5
#     5-fold: k=5
#     6-fold: k=3
# Cubic (L3 norm)
#     3-fold: k=3
#     4-fold: k=5
#     5-fold: k=5
#     6-fold: k=5 

5

In [13]:
max_value

0.9603174603174603

### Fitting kNN on PCA-transformed data:

In [14]:
#load in design matrix:
dat = np.load("dat_transform.npz")

#extract design matrix from npz file:
dat = dat["arr_0"]   # (25231,280)

In [15]:
#load in response vector:
response = np.load("response.npz")

#extract response vector from npz file:
response = response["arr_0"]   

In [16]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(
    dat, response, test_size=0.4, random_state=0
)

In [17]:
np.shape(train_X) # (20184, 500)
np.shape(test_X) # (5047, 500)
np.shape(train_y) # (20184,)
np.shape(test_y) # (5047,)

(10093,)

### Tuning k for our Transformed Data (We consider k = 1,3,5,7,9)
* We consider folds: 3,4,5,6 to retain as many observations within the validation training and testing sets as possible 
- testing set size = [3364,6728], training set size > testing set size.

|Distance Metric | Folds | Optimal k |    Average Validation Accuracy  |          Run time            |
|----------------|-------|-----------|---------------------------------|------------------------------|
|Manhattan       |   3   |     9     |          38.876%                |    5488.2107247 seconds      |
|Euclidean       |   3   |     9     |          38.935%                |    5716.17 seconds           |

In [20]:
import timeit
timestart = timeit.default_timer()           # start timer

optimal_k,max_value = k_tune(train_X,train_y,distance_type = 'manhattan',folds=3)

timeend = timeit.default_timer()             # end timer
time_elapsed = (timeend - timestart) # calculate elapsed time

print("The optimal number of neighbors for Manhattan distances with 3-folds is: ",optimal_k,", with validation accuracy: ",max_value)
print("Run time is: ",time_elapsed)

The optimal number of neighbors for Manhattan distances with 3-folds is:  9 , with validation accuracy:  0.38875677103976747
Run time is:  5488.2107247


In [21]:
timestart = timeit.default_timer()           # start timer

optimal_k,max_value = k_tune(train_X,train_y,distance_type = 'euclidean',folds=3)

timeend = timeit.default_timer()             # end timer
time_elapsed = (timeend - timestart) # calculate elapsed time

print("The optimal number of neighbors for Euclidean distances with 3-folds is: ",optimal_k,", with validation accuracy: ",max_value)
print("Run time is: ",time_elapsed)

The optimal number of neighbors for Euclidean distances with 3-folds is:  9 , with validation accuracy:  0.38935130136081386
Run time is:  5716.176610099999


### Test Accuracy:
|Distance Metric |         k |           Test Accuracy         |          Run time            |
|----------------|-----------|---------------------------------|------------------------------|
|Manhattan       |     9     |          39.45%                 |    1136.14 seconds           |
|Euclidean       |     9     |          39.08%                 |    1157.30 seconds           |

In [26]:
timestart = timeit.default_timer()           # start timer

mahattan_9 = predict(train_X,train_y,test_X,test_y,k=9,distance_type='manhattan')
timeend = timeit.default_timer()             # end timer
time_elapsed = (timeend - timestart) # calculate elapsed time

print("Prediction accuracy for Manhattan distances is: ",mahattan_9)
print("Run-time for Manhattan distances is: ",time_elapsed)

Prediction accuracy for Manhattan distances is:  0.39453086297433865
Run-time for Manhattan distances is:  1136.135374900001


In [29]:
timestart = timeit.default_timer()           # start timer

euclidean_9 = predict(train_X,train_y,test_X,test_y,k=9,distance_type='euclidean')
timeend = timeit.default_timer()             # end timer
time_elapsed = (timeend - timestart) # calculate elapsed time

print("Prediction accuracy for Manhattan distances is: ",euclidean_9)
print("Run-time for Manhattan distances is: ",euclidean_9)

Prediction accuracy for Manhattan distances is:  0.39086495591003667
Run-time for Manhattan distances is:  1157.3048200999983
